In [19]:
import numpy as np
import copy
from tqdm import tqdm

In [20]:
def get_action(phi, k, l):
    return np.sum(-2 * k * phi * (np.roll(phi, 1, 0) + np.roll(phi, 1, 1))
                  + (1 - 2 * l) * phi**2 + l * phi**4)

def get_drift(phi, k, l):
    return (2 * k * (np.roll(phi, 1, 0) + np.roll(phi, -1, 0)
                     + np.roll(phi, 1, 1) + np.roll(phi, -1, 1))
            + 2 * phi * (2 * l * (1 - phi**2) - 1))

def get_hamiltonian(chi, action):
    return 0.5 * np.sum(chi**2) + action

In [21]:
def hmc(phi_0, S_0, k, l, n_steps=100):
    dt = 1 / n_steps

    phi = phi_0
    chi = np.random.randn(*phi.shape)
    H_0 = get_hamiltonian(chi, S_0)

    chi += 0.5 * dt * get_drift(phi, k, l)
    for i in range(n_steps-1):
        phi += dt * chi
        chi += dt * get_drift(phi, k, l)
    phi += dt * chi
    chi += 0.5 * dt * get_drift(phi, k, l)

    S = get_action(phi, k, l)
    dH = get_hamiltonian(chi, S) - H_0

    if dH > 0:
        if np.random.rand() >= np.exp(-dH):
            return phi_0, S_0, False
    return phi, S, True

In [22]:
N = 32
k = 0.3
l = 0.02

phi = np.random.randn(N,N)
S = get_action(phi, k, l)

In [23]:
for _ in tqdm(range(1000)):
    phi, S, accepted = hmc(phi, S, k, l)

100%|██████████| 1000/1000 [00:07<00:00, 141.48it/s]


In [24]:
cfgs = []

for i in tqdm(range(1000)):
    phi, S, accepted = hmc(phi, S, k, l)
    
    if i % 10 == 0:
        cfgs.append(copy.deepcopy(phi))
        
cfgs = np.array(cfgs)

100%|██████████| 1000/1000 [00:07<00:00, 142.85it/s]
